In [36]:
spark.sql(
    '''
       CREATE TABLE IF NOT EXISTS txkt.dws_video
       (
         `video_id`             STRING COMMENT '视频ID',
         `video_title`          STRING COMMENT '视频标题',
         `add_study_num_1d`     BIGINT COMMENT '课程近一天增加学习人数',
         `add_study_num_3d`     BIGINT COMMENT '课程近三天增加学习人数',
         `add_study_num_10d`    BIGINT COMMENT '课程近十天增加学习人数', 
         `add_person_num_1d`    BIGINT COMMENT '课程近一天增加购买/报名人数',
         `add_person_num_3d`    BIGINT COMMENT '课程近三天增加购买/报名人数',
         `add_person_num_10d`   BIGINT COMMENT '课程近十天增加购买/报名人数', 
         `profit_num_td`        BIGINT COMMENT '课程累计收益（元）',
         `profit_num_1d`        BIGINT COMMENT '课程近一天收益（元）',
         `profit_num_3d`        BIGINT COMMENT '课程近三天收益（元）',
         `profit_num_10d`       BIGINT COMMENT '课程近十天收益（元）', 
         `etl_time`             STRING COMMENT '数据加工时间'      
       ) 
       PARTITIONED BY (`dt` STRING) 
       STORED AS PARQUET
    '''
)


DataFrame[]

In [44]:
spark.sql(
    '''
       INSERT OVERWRITE TABLE txkt.dws_video PARTITION (dt = "{dt}")
            SELECT today.video_id,
                   today.video_title,
                   (NVL(today.recently_study_num, 0) -  NVL(d1.recently_study_num, 0))          AS add_study_num_1d,
                   (NVL(today.recently_study_num, 0) -  NVL(d3.recently_study_num, 0))          AS add_study_num_3d,
                   (NVL(today.recently_study_num, 0) - NVL(d10.recently_study_num, 0))          AS add_study_num_10d,
                   (NVL(today.person_num, 0) -  NVL(d1.person_num, 0))                          AS add_person_num_1d,
                   (NVL(today.person_num, 0) -  NVL(d3.person_num, 0))                          AS add_person_num_3d,
                   (NVL(today.person_num, 0) - NVL(d10.person_num, 0))                          AS add_person_num_10d,
                   today.person_num                                    * today.price            AS profit_num_td,
                   (NVL(today.person_num, 0) -  NVL(d1.person_num, 0)) * today.price            AS profit_num_1d,
                   (NVL(today.person_num, 0) -  NVL(d3.person_num, 0)) * today.price            AS profit_num_3d,
                   (NVL(today.person_num, 0) - NVL(d10.person_num, 0)) * today.price            AS profit_num_10d,
                   FROM_UNIXTIME(CAST(NOW() AS BIGINT), 'yyyy-MM-dd HH:mm:ss')                  AS elt_time
              FROM (
                     SELECT video_id,
                            video_title,
                            recently_study_num, 
                            person_num,
                            price
                       FROM txkt.dim_video_df
                      WHERE dt = '{dt}'
                   ) AS today
         LEFT JOIN (
                     SELECT video_id,
                            video_title,
                            recently_study_num,
                            person_num,
                            price
                       FROM txkt.dim_video_df
                      WHERE dt = DATE_SUB('{dt}',1)
                   ) AS d1
                ON today.video_id = d1.video_id
         LEFT JOIN (
         
            
                     SELECT video_id,
                            video_title,
                            recently_study_num,
                            person_num,
                            price
                       FROM txkt.dim_video_df
                      WHERE dt = DATE_SUB('{dt}', 3)
                   ) AS d3
                ON today.video_id = d3.video_id
         LEFT JOIN (
                     SELECT video_id,
                            video_title,
                            recently_study_num,
                            person_num,
                            price
                       FROM txkt.dim_video_df
                      WHERE dt = DATE_SUB('{dt}', 10)
                   ) AS d10
                ON today.video_id = d10.video_id
    '''.format(dt='2022-03-26')
)

DataFrame[]